In [60]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import json
import os
import pickle
import EarlyExitModel
import torch.nn as nn

In [3]:
# Check if PyTorch is using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch is using: {device}")

# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Test a simple PyTorch tensor operation on the GPU
x = torch.tensor([2.0, 3.0], device=device)
y = torch.tensor([4.0, 5.0], device=device)
result = x * y
print(f"Result of tensor operation: {result}")

PyTorch is using: cuda
PyTorch version: 2.2.1+cu121
Result of tensor operation: tensor([ 8., 15.], device='cuda:0')


In [4]:
orgmodel = models.resnet50(pretrained=True)
last_layer = orgmodel.fc

c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\afxwo\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
orgmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
last_layer

Linear(in_features=2048, out_features=1000, bias=True)

In [6]:
with open("imagenet_class_index.json", "r") as jf:
    class_index_json = json.load(jf)
print(class_index_json)
class_indices = {}
for k, v in class_index_json.items():
    class_indices[v[0]] = int(k)
print(class_indices)

{'0': ['n01440764', 'tench'], '1': ['n01443537', 'goldfish'], '2': ['n01484850', 'great_white_shark'], '3': ['n01491361', 'tiger_shark'], '4': ['n01494475', 'hammerhead'], '5': ['n01496331', 'electric_ray'], '6': ['n01498041', 'stingray'], '7': ['n01514668', 'cock'], '8': ['n01514859', 'hen'], '9': ['n01518878', 'ostrich'], '10': ['n01530575', 'brambling'], '11': ['n01531178', 'goldfinch'], '12': ['n01532829', 'house_finch'], '13': ['n01534433', 'junco'], '14': ['n01537544', 'indigo_bunting'], '15': ['n01558993', 'robin'], '16': ['n01560419', 'bulbul'], '17': ['n01580077', 'jay'], '18': ['n01582220', 'magpie'], '19': ['n01592084', 'chickadee'], '20': ['n01601694', 'water_ouzel'], '21': ['n01608432', 'kite'], '22': ['n01614925', 'bald_eagle'], '23': ['n01616318', 'vulture'], '24': ['n01622779', 'great_grey_owl'], '25': ['n01629819', 'European_fire_salamander'], '26': ['n01630670', 'common_newt'], '27': ['n01631663', 'eft'], '28': ['n01632458', 'spotted_salamander'], '29': ['n01632777', 

In [7]:
imagenette_classes = [0, 217, 482, 491, 497, 566, 569, 571, 574, 701]
update_class_indices = {}
for i in range(len(imagenette_classes)):
    update_class_indices[imagenette_classes[i]] = i
print(update_class_indices)

{0: 0, 217: 1, 482: 2, 491: 3, 497: 4, 566: 5, 569: 6, 571: 7, 574: 8, 701: 9}


In [9]:
with open("..\\ee_models\\resnet_alpha075.pkl", "rb") as f:
    model075 = pickle.load(f)
model075.eval()
model075.to(device)

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): OptionalExitModule(
      (module): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [40]:
temp = Image.open("..\\torch_normal_adv_samples\\results_transfer\\outputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)
with torch.no_grad():
    tempout = model075(temp)
    print(tempout)
    temppredclass = torch.argmax(tempout, dim=1).item()
print(temppredclass)


tensor([[-16822.9531, -16823.4062, -16913.8926, -16835.8535, -16856.2266,
         -16882.8125, -16907.2891, -16863.5410, -16872.7656, -16911.5762]],
       device='cuda:0')
0


In [19]:
def get_ee_classification(classname, filename, model):
    image = Image.open(f"..\\torch_normal_adv_samples\\results_transfer\\outputs\\{classname}\\{filename}")
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        predclass = torch.argmax(output, dim=1).item()
    return predclass

In [25]:
def get_ee_classification_set(directory_path, model):
    classfolders = [fol for fol in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, fol))]
    for classname in classfolders:
        expected = update_class_indices[class_indices[classname]]
        cnt = 0
        correct = 0
        # print(classname)
        sdpath = os.path.join(directory_path, classname)
        files = [os.path.join(sdpath, fil) for fil in os.listdir(sdpath)]
        for file in files:
            filename = file.split("\\")[-1]
            eec = get_ee_classification(classname, filename, model)
            if expected == eec:
                correct += 1
            cnt += 1
        print(f"Class: {classname} has got accuracy of {correct / cnt * 100}%")
            

In [26]:
# with alpha = 0.75 model
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer\\outputs", model075)

tensor([[-52.8601, -79.5434, -75.9896, -81.3551, -79.2835, -78.2073, -75.3502,
         -78.9040, -79.2891, -81.9402]], device='cuda:0')
tensor([[-47.5320, -78.1771, -77.2189, -77.4546, -73.6466, -75.9219, -80.2771,
         -79.8785, -82.5942, -79.6048]], device='cuda:0')
tensor([[ -66.3998,  -94.2035,  -98.3464, -103.6556,  -96.9234,  -98.6452,
          -95.0345,  -91.6456,  -98.5464,  -94.7164]], device='cuda:0')
tensor([[-55.4665, -83.1903, -83.6980, -84.2922, -89.3515, -89.8405, -86.0942,
         -82.4067, -85.4348, -87.0331]], device='cuda:0')
tensor([[-64.3379, -94.8342, -92.4719, -99.0503, -96.9462, -92.4189, -94.6388,
         -94.0480, -97.0052, -95.9406]], device='cuda:0')
tensor([[ -75.4316, -109.7291, -110.4411, -103.2524, -110.9005, -109.3376,
         -109.9235, -106.5174, -104.5240, -116.0683]], device='cuda:0')
Class: n01440764 has got accuracy of 80.0%
tensor([[ -95.8262,  -65.1802,  -96.2365,  -96.8477, -100.9924,  -97.0075,
          -96.6581,  -97.1327,  -98.3779

In [27]:
with open("..\\ee_models\\resnet_alpha025.pkl", "rb") as f:
    model025 = pickle.load(f)
model025.eval()
model025.to(device)

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): OptionalExitModule(
      (module): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [28]:
# with alpha = 0.25 model
get_ee_classification_set("..\\torch_normal_adv_samples\\results_transfer\\outputs", model025)

tensor([[-51.2113, -84.7534, -83.3414, -89.3655, -81.7619, -85.9657, -89.0956,
         -90.5682, -90.1499, -87.3126]], device='cuda:0')
tensor([[-60.1787, -90.7875, -86.2181, -89.2126, -92.3275, -88.8593, -86.2787,
         -91.6708, -95.4922, -92.2458]], device='cuda:0')
tensor([[ -90.2622, -119.6905, -120.1763, -115.7269, -124.4942, -124.7710,
         -129.2516, -121.0058, -120.1605, -123.5976]], device='cuda:0')
tensor([[ -69.3111,  -99.3507, -102.1606,  -97.5343, -102.3994, -100.3316,
         -100.5952, -101.1954,  -99.2173, -109.3618]], device='cuda:0')
tensor([[-54.4320, -86.6656, -84.7478, -93.0649, -85.1041, -88.8659, -86.7840,
         -89.8858, -88.7066, -90.5413]], device='cuda:0')
tensor([[-63.3990, -92.1487, -91.5760, -92.5287, -98.0431, -93.2405, -91.1837,
         -87.9441, -92.0958, -99.8722]], device='cuda:0')
tensor([[-60.7059, -94.7196, -96.3225, -92.3934, -97.7497, -94.1414, -93.4796,
         -94.6527, -94.4322, -99.4761]], device='cuda:0')
tensor([[-56.6634, -8

In [46]:
temp = Image.open("..\\torch_normal_adv_samples\\results_transfer\\outputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
temp = transform(temp)
temp = temp.unsqueeze(0).to(device)

In [41]:
x = list(model025.model.children())[:4]

In [49]:
output = temp
for layer in x:
    layer.to(device)
    output = layer(output)

output = torch.flatten(output, start_dim=1)

In [50]:
output.shape

torch.Size([1, 200704])

In [55]:
y = list(list(model025.model.children())[4].children())[1:2]

In [56]:
y

[Linear(in_features=200704, out_features=1, bias=True)]

In [58]:
output = y[0](output)

In [59]:
output

tensor([[-0.2870]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [89]:
class ThresholdLayer(nn.Module):
    def __init__(self):
        super(ThresholdLayer, self).__init__()

    def forward(self, x):
        return torch.where(x >= 0.5, torch.ones_like(x), torch.zeros_like(x))

In [97]:
layers = x + [nn.Flatten()] + y # + [ThresholdLayer()]
smallmodel = nn.Sequential(*layers)

In [98]:
smallmodel

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): Linear(in_features=200704, out_features=1, bias=True)
)

In [99]:
smallmodel(temp)

tensor([[-0.2870]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [142]:
def generate_adversaries(model, baseImage, delta, classIdx, steps=50, epsilon=0.005):
    for step in range(steps):
        # watch gradients
        baseImage.requires_grad = True
        delta.requires_grad = True
        
        # add perturbation
        adversary = baseImage + delta
        
        # run perturbed image through the model and calculate the loss
        predictions = model(adversary)
        print(predictions)
        # if step % 5 == 0:
        #     print("Step: {}, Pred: {}..., Confidence: {}".format(step, torch.argmax(predictions, 1).item(), torch.softmax(predictions, dim=1)[0][torch.argmax(predictions, 1).item()].item() * 100))
        # loss = torch.nn.CrossEntropyLoss()(predictions, torch.tensor([classIdx]).to(device))
        # print(predictions)
            
        # loss_classification = torch.nn.CrossEntropyLoss()(predictions, torch.tensor([classIdx]).to(device))
        # print(loss_classification)
        
        # Calculate regularization term to move output towards target_value
        deviation = predictions - .75
        # print(deviation)
        sign = torch.sign(deviation)
        loss_regularization = torch.abs(deviation) * sign  # Penalize deviation from target_value
        
        # Total loss
        loss = 0.1 * loss_regularization.mean()
        
        # log loss
        if step % 5 == 0:
            print("Step: {}, Loss: {}...".format(step, loss.item()))
        
        # calculate gradients of loss with respect to perturbation
        model.zero_grad()
        loss.backward()
        
        # update the perturbation vector
        delta_grad = delta.grad.detach().sign()  # take sign of the gradient
        delta.data = (delta + epsilon * delta_grad).clamp(-epsilon, epsilon)  # update delta and clip
        
        # clamp the perturbation vector
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
    return delta

In [101]:
image = Image.open("..\\torch_normal_adv_samples\\results_transfer\\outputs\\n01440764\\ILSVRC2012_val_00007197.JPEG")
image = transform(image)
image = image.unsqueeze(0).to(device)

In [123]:
image.shape

torch.Size([1, 3, 224, 224])

In [143]:
image.requires_grad = True
delta = torch.zeros_like(image, requires_grad=True).to(device)

classIdx = 0
adversary = generate_adversaries(smallmodel, image, delta, classIdx, steps=20, epsilon=0.005)

tensor([[-0.2870]], device='cuda:0', grad_fn=<AddmmBackward0>)
Step: 0, Loss: -0.10369708389043808...
tensor([[0.5040]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.5813]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6231]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6442]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6571]], device='cuda:0', grad_fn=<AddmmBackward0>)
Step: 5, Loss: -0.009287172928452492...
tensor([[0.6634]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6673]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6708]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6728]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6741]], device='cuda:0', grad_fn=<AddmmBackward0>)
Step: 10, Loss: -0.007589865010231733...
tensor([[0.6756]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6769]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.6774]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.

In [144]:
adversary_image = (image + adversary).clamp(0, 1).squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_image = (adversary_image * 255).astype('uint8')
adversary_image = Image.fromarray(adversary_image)

In [145]:
smallmodel.to(device)
new = transform(adversary_image).unsqueeze(0).to(device)
output = smallmodel(new)
print(output)

tensor([[0.8312]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [135]:
with torch.no_grad():
    output = model025(new)
    predclass = torch.argmax(output, dim=1).item()

In [136]:
predclass

3

In [137]:
adversary_image.save('output.JPEG')

In [138]:
adversary_pil = adversary.squeeze().permute(1, 2, 0).cpu().detach().numpy()
adversary_pil = Image.fromarray((adversary_pil * 255).astype(np.uint8))

In [139]:
adversary_pil.save("change.JPEG")